In [21]:
import sys
import logging
import boto3
from datetime import datetime
from datetime import datetime, timedelta
import math
import random
import numpy as np
import json

from datetime import datetime
import boto3
import json
import pandas as pd
from datetime import datetime, timedelta
import boto3
import warnings
import pandas as pd
import time
import traceback
import warnings
import decimal
from decimal import Decimal
import json

import re
##libraries required
import math
import re
import random
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy
import csv
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import pandas as pd
from pathlib import Path
from tokenizers.trainers import BpeTrainer,WordPieceTrainer
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder,WordPiece as WordPieceDecoder
from tokenizers.models import BPE,WordPiece
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import BertPreTokenizer


In [22]:
pd.read_csv('item-tags-sg-9may.csv').head()

,country_id,name,department_name,sub_department_name
0,4.0,Mysore Sandal Powder,Personal Care,Feminine Care
1,4.0,Life Alkaline Bottle Water pH 8.5 510ml,Beverages,Water
2,4.0,Ben & Jerry Choc Fud Brownie 458ml [68030002],Ice Cream & Dessert,Ice Cream
3,4.0,NN016 BoBo Cooked Fishball 250g (26 pcs) I 250 g,Eid al-Fitr,Chilled/Frozen
4,4.0,"2 x Tullamore D.E.W. Original, 700ml","Wine, Beers & Spirits",Spirits


# TRAIN TOKENIZER

## Byte Level BPE

In [25]:
# USed in Roberta Model
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("item-tags-sg-9may.csv")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=1_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])


In [27]:
encoding = tokenizer.encode("Mysore Sandal Powder")
encoding.tokens

['M', 'y', 's', 'ore', 'ĠS', 'and', 'al', 'ĠPowder']

None


## Sentence Piece

In [41]:
import sentencepiece as spm
# Train Encoder
spm.SentencePieceTrainer.train('--input=item-tags-sg-9may.csv --model_prefix=m_ind --vocab_size=5000')

In [42]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m_ind.model')

True

In [43]:
print(sp.encode_as_pieces('This is a test'))

['▁Th', 'is', '▁', 'is', '▁', 'a', '▁', 't', 'est']


In [44]:
print(sp.encode_as_ids('This is a test'))

[1959, 702, 4, 702, 4, 71, 4, 82, 1342]


## Word Piece

In [46]:


# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = Tokenizer(WordPiece())

# Then we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be
# executed in order.
tokenizer.normalizer = Sequence([
    NFKC(),
    Lowercase()
])

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = BertPreTokenizer()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = WordPieceDecoder()


paths = [str(x) for x in Path(".").glob("item-tags-sg-9may.csv")]



In [47]:


# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = WordPieceTrainer(vocab_size=30522, show_progress=True,special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],wordpieces_prefix="##")
tokenizer.train(files=paths, trainer=trainer)

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 14476


In [51]:
encoding = tokenizer.encode("This is a test!")
encoding.tokens

['th', '##is', 'is', 'a', 'test', '!']

In [58]:
encoding.ids

[1348, 1150, 3763, 36, 8227, 5]

In [17]:
!mkdir ItemTaggingMultiLingualWordPiece

In [18]:
tokenizer.model.save("ItemTaggingMultiLingualWordPiece")

['ItemTaggingMultiLingualWordPiece/vocab.txt']

In [50]:
!cat ItemTaggingMultiLingualWordPiece/vocab.txt

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
!
"
#
$
%
&
'
(
)
*
+
,
-
.
/
0
1
2
3
4
5
6
7
8
9
:
;
>
[
]
_
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z
|
~
®
°
×
à
â
ã
è
é
ñ
ö
ø
ā
π
‏
–
’
‚
”
•
€
⏫
⭐
【
】
あ
え
く
こ
ご
た
だ
と
ま
め
り
ろ
ん
イ
キ
ク
ケ
サ
ス
ソ
ッ
テ
ト
ホ
ミ
ー
一
丁
三
下
不
与
东
丝
中
丸
丹
丽
乌
乐
九
乳
云
五
亚
仁
仙
仲
份
伤
余
你
便
修
停
健
傅
儿
光
克
全
八
公
关
具
养
军
农
冠
冬
冰
冷
冻
凉
凝
出
刀
切
初
制
剂
前
力
助
包
化
医
十
千
升
半
华
南
卷
原
叁
参
口
古
可
台
叶
司
合
吉
味
和
咖
咚
咸
品
哟
啡
喉
喱
嗽
囊
四
国
圆
圈
土
圭
地
块
坚
坡
堡
塑
壁
士
复
外
多
大
天
太
夫
头
夹

奶
威
娘
婆
婴
子
安
官
宝
容
宾
小
少
尿
山
峇
州
巢
工
巧
巴
巾
布
师
帝
带
干
平
年
幸
幼
广
康
强
当
彩
律
德
心
怀
恋
惹
手
打
抖
护
拉
拜
排
撤
支
散
敲
料
断
斯
新
方
旋
无
日
旧
早
旺
昌
星
晋
景
晶
暖
曼
月
有
木
末
本
杏
杜
杞
条
来
杨
杯
松
板
林
果
枝
枣
枸
柑
柔
柚
柠
柿
标
核
桃
梅
梨
棉
森
椒
椰
榴
樱
橙
檬
止
毒
比
毛
毫
水
永
汁
汉
汤
沐
沙
油
泉
泡
泰
洁
洋
洗
洲
浆
浓
浴
浸
消
润
液
淇
淋
混
清
湖
湾
湿
滋
漳
澄
澳
火
炒
炙
炸
点
烤
烧
热
焦
然
熊
熟
燕
片
牌
牛
特
犁
狗
独
猪
猫
玉
王
玩
环
玻
珍
珠
球
瑞
璃
瓜
瓶
甜
生
番
痘
痛
白
百
的
皮
益
盒
直
真
眼
睫
矿
碎
碧
碰
礼
神
秋
科
空
窝
立
章
童
竭
笔
答
米
粉
粒
粮
精
糕
糖
素
紫
红
级
纯
纷
纸
线
组
细
维
绿
缤
罐
羊
美
羚
翘
翠
翡
老
耳
肉
肠
肾
肿
胡
胶
脆
腐
腰
腿
膏
色
芋
芒
芝
芥
芭
花
苏
苓
苹
